In [5]:
import pandas as pd

dados = pd.read_csv(r"C:\Users\User\Desktop\trabalho_calculo\data\calculus_data.csv")
dados.head()

,time,ax,ay,az,aT
0,0.005,-0.0112,0.0159,0.0539,0.057
1,0.023,-0.0721,-0.0435,-0.1818,0.200
2,0.040,0.0073,-0.0086,-0.1048,0.105
3,0.059,-0.1270,0.0126,-0.1444,0.193
4,0.078,0.0337,0.0125,0.1407,0.145
